In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from scipy.signal import butter, filtfilt

In [ ]:
def map_motion_percent (df):
    
    motion_table_file_path = 'motion_sequence.csv'
    motion_df = pd.read_csv(motion_table_file_path)

    motion_values =[]
    
    df['Angle_int'] = df['Angle'].astype(int)

    # 데이터프레임1과 데이터프레임2를 동시에 비교
    for _, row1 in df.iterrows():
        angle1 = row1['Angle_int']
        state1 = row1['State']
    
        matching_row = motion_df[(motion_df['Angle'] == angle1) & (motion_df['State'] == state1)]
        if not matching_row.empty:
            motion_value = matching_row['Motion'].values[0]
            motion_values.append(motion_value)
        else: 
            motion_value = np.nan
            motion_values.append(motion_value)


    df['Motion'] = motion_values
    df['Motion'] = df['Motion'].fillna((df['Motion'].ffill() + df['Motion'].bfill()) / 2)

    return df

In [ ]:
def combined_torques_graph(df, df_evaluate):
    df_evaluate = map_motion_percent(df_evaluate)
    df = df.sort_values('Motion')
    df_evaluate = df_evaluate.sort_values('Motion')

    plt.figure(figsize=(15, 10))

    # 첫 번째 서브플롯 (상단)
    plt.subplot(2, 1, 1)  # 2행 1열로 배치, 첫 번째 서브플롯
    
    plt.plot(df['Motion'], df['InteractionTorque'], label='Interaction Torque', color='green')
    plt.plot(df['Motion'], df['MotorTorque'], label='Motor Torque', color='blue')
    plt.plot(df['Motion'], df['HumanTorque'],  label='Human Torque',color='red')

    plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
    plt.ylim(-3, 2)
    plt.xlabel('A motion (%)')
    plt.ylabel('Torques (N.m)')
    plt.legend()

    # 두 번째 서브플롯 (하단)
    plt.subplot(2, 1, 2)  # 2행 1열로 배치, 두 번째 서브플롯
    plt.plot(df_evaluate['Motion'], df_evaluate['New_Range_0'], label='0', color='grey')
    plt.plot(df_evaluate['Motion'], df_evaluate['New_Range_1'], label='1', color='grey')
    plt.plot(df['Motion'], df['HumanTorque'], label='Human Torque', color='red')

    plt.grid(True, color='gray', linestyle='--', linewidth=0.5)
    plt.ylim(-7, 2)
    plt.xlabel('A motion (%)')
    plt.ylabel('Human Torque (N.m)')
    plt.legend()

    plt.tight_layout()  # 서브플롯 간의 간격 조정
    plt.show()

In [ ]:
def compare_value(df): 
    df['HumanTorque'] = df['HumanTorque'].abs()
    average_value = df['HumanTorque'].mean()
    return average_value

In [ ]:
def main():

    file_path = 'interaction_control_node_1.csv'
    df = pd.read_csv(file_path)
    evaluate_file_path = 'Evaluation_Table2.csv' 
    df_evaluate = pd.read_csv(evaluate_file_path)
    
    #N번 사람 비교
    total_score = compare_value(df)
    
    
    print('Interactivitiy Score is '+ str(total_score))
    
    #inter_force_graph (df)
    #torques_graph(df_number)
    combined_torques_graph (df, df_evaluate)

In [ ]:
if __name__ == '__main__':
    main()
